In [146]:
import os

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import cv2
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.preprocessing import label_binarize
from itertools import cycle
import splitfolders


if str(os.getcwdb()[-3:]).split("'")[1] != 'src':
    for _ in range(2):
        os.chdir(os.path.dirname(os.getcwdb()))


# Proceso

- Se borran las imágenes vacías (".png") y las que se eliminaron durante el procesado del "dataframe" de características, y se actualiza ese conjunto de datos quitando esas imágenes sin diamantes

- El resultado es que en la carpeta de imágenes quedan el mismo número de archivos ".jpg" que en el índice del "dataframe" de características

In [147]:
df_images_data = pd.read_csv(r'data\processed\images_data_processed.csv', index_col='Id')
raw_data_path = r'data\raw\images'


In [148]:
df_images_data

,weight (carat),cut quality,color quality,clarity quality,depth (percentage),lenght (millimeters),width (millimeters),depth (millimeters)
Id,,,,,,,,
1638147,0.55,4.0,5.0,1.0,62.553191,5.05,4.35,2.94
1612606,0.51,4.0,2.0,3.0,64.900662,4.71,4.35,2.94
1638140,0.50,4.0,2.0,3.0,62.813522,4.91,4.26,2.88
1536093,0.53,4.0,6.0,2.0,65.720524,4.70,4.46,3.01
1643527,0.52,4.0,1.0,6.0,65.141612,4.76,4.42,2.99
...,...,...,...,...,...,...,...,...
1624337,0.50,4.0,3.0,3.0,60.523763,5.14,5.17,3.12
1751206,0.50,4.0,5.0,2.0,63.346614,5.00,5.04,3.18
1773024,0.50,4.0,5.0,2.0,62.636274,5.03,5.06,3.16


In [149]:
# Hay varias imágenes vacías. Todas están en formato "png". Hay que eliminarlas antes que nada
no_delete_extension = '.jpg'
for (dirpath, dirnames, filenames) in os.walk(raw_data_path):
    for file in filenames:
        if no_delete_extension not in file:
            if file != 'info_raw.txt':
                os.remove(f'{dirpath}/{file}')


In [150]:
# Se han eliminado muchas filas del "dataframe", con lo que toca borrar las imágenes correspondientes
for (dirpath, dirnames, filenames) in os.walk(raw_data_path):
    for file in filenames:
        if file[:-4] not in df_images_data.index and file[:-4] != 'info_raw':
            os.remove(f'{dirpath}/{file}')


In [151]:
# Se crea una lista de las imágenes definitivas de la carpeta
filenames_list = []
for (dirpath, dirnames, filenames) in os.walk(raw_data_path):
    filenames_list += [file[:-4] for file in filenames if file != "info_raw.txt"]

len(filenames_list)


3913

In [152]:
# Asimismo, hay que borrar del "dataframe" los diamantes que se han sacado de la carpeta por no tener imagen
for index in df_images_data.index:
    if index not in filenames_list:
        try:
            df_images_data.drop(index, inplace=True)
        except Exception:
            print(f'El diamante {index} ya se borró durante la limpieza')

len(df_images_data)


3913

In [153]:
# En el "dataframe" de características de imagen solo quedan los mismos que en la carpeta de imágenes "raw".
sorted(df_images_data.index) == sorted(filenames_list)


True

In [154]:
# En el "dataframe" de características de imagen solo quedan los mismos que en la carpeta de imágenes "raw". Se actualiza el archivo procesado
df_images_data.to_csv(r'data\processed\images_data_processed.csv', index='Id')
